In [1]:
import os

import delta
import mimesis
import pyspark

from pyspark.sql import SparkSession

In [2]:
def get_spark() -> SparkSession:
    builder = (
        pyspark.sql.SparkSession.builder.appName("TestApp")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
    )
    spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

In [3]:
def create_dataset(i: int = 100) -> list[dict]:
    fake = mimesis.Generic()
    output = [
        {
            "name": fake.person.name(),
            "surname": fake.person.surname(),
            "birthday": fake.datetime.date(1960, 2010),
            "email": fake.person.email(),
            "country": fake.address.country(),
            "state": fake.address.state(),
            "city": fake.address.city(),
        }
        for _ in range(i)
    ]
    return output


In [4]:
def main():
    dir = os.getcwd()
    spark = get_spark()

    df = spark.createDataFrame(create_dataset(i=1_000_000))

    df = df.select(
        df.name, df.surname, df.birthday, df.email, df.country, df.state, df.city
    )
    df.write.format("delta").mode("overwrite").save(dir + os.sep + "people")
    df.show()
    

In [5]:
if __name__ == "__main__":
    main()


24/02/17 15:05:52 WARN Utils: Your hostname, MacBookJulio.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
24/02/17 15:05:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/dal/anaconda3/envs/delta_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dal/.ivy2/cache
The jars for the packages stored in: /Users/dal/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2eec8c81-1b94-46dc-b788-3908b4ced951;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 236ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

24/02/17 15:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+--------+--------+----------+--------------------+-----------------+------------+----------------+
|    name| surname|  birthday|               email|          country|       state|            city|
+--------+--------+----------+--------------------+-----------------+------------+----------------+
| Matthew|    Reed|2005-10-21| above1981@gmail.com|           Malawi|     Alabama|         Norwich|
|  Golden|  Jordan|2008-02-08|elements1935@gmai...|     Sierra Leone|      Hawaii|          Edmond|
| Dinorah|   White|1991-12-12| andy1850@yandex.com|          Eritrea|    Missouri|           Sandy|
|   Myung|   Blair|1970-02-22|donate1938@outloo...|    United States|     Alabama|    West Chester|
|   Thanh| Wiggins|1976-11-27|advance1823@yande...|            Chile|  New Jersey|     Perth Amboy|
| Lashell| Johnson|2007-07-04|supervision1951@o...|           Guyana|    Oklahoma|         Roselle|
| Pandora|Sandoval|1990-07-23| net1956@outlook.com|      Timor-Leste|North Dakota|       Montclair|


#### setup env no OSX
https://www.linkedin.com/pulse/set-up-pyspark-delta-lake-your-local-machine-marius-krol/